In [ ]:
# 🏗️ 건축 도면 RAG 질의 시스템
# CLI 도구(query_rag.py)와 노트북 통합 인터페이스

import subprocess
import json
import os
import time
import re
from PIL import Image
import matplotlib.pyplot as plt

def run_rag_query(query: str, project: str = None, n_results: int = 3, use_llm: bool = True):
    """
    CLI query_rag.py 도구를 직접 호출하여 RAG 질의를 수행합니다.
    
    Args:
        query (str): 검색할 질의 내용
        project (str, optional): 특정 프로젝트명 (None이면 모든 프로젝트 검색)
        n_results (int): 반환할 검색 결과 수
        use_llm (bool): LLM 답변 생성 여부
    
    Returns:
        str: CLI 도구의 출력 결과
    """
    cmd = ["python", "query_rag.py", query, "-n", str(n_results)]
    
    if project:
        cmd.extend(["-p", project])
    
    if not use_llm:
        cmd.append("--no_llm")
    
    try:
        print(f"🔍 CLI 실행: {' '.join(cmd)}")
        result = subprocess.run(
            cmd, 
            capture_output=True, 
            text=True, 
            cwd="/Users/joguk/VLM"
        )
        
        if result.returncode == 0:
            print("✅ CLI 실행 성공")
            return result.stdout
        else:
            print(f"❌ CLI 실행 실패 (코드: {result.returncode})")
            print(f"에러: {result.stderr}")
            return f"오류: {result.stderr}"
            
    except Exception as e:
        print(f"❌ CLI 실행 중 예외 발생: {e}")
        return f"예외: {str(e)}"

def get_available_projects():
    """사용 가능한 프로젝트 목록을 CLI 도구로 조회합니다."""
    cmd = ["python", "query_rag.py", "--list_projects"]
    
    try:
        result = subprocess.run(
            cmd, 
            capture_output=True, 
            text=True, 
            cwd="/Users/joguk/VLM"
        )
        
        if result.returncode == 0:
            return result.stdout
        else:
            return f"오류: {result.stderr}"
            
    except Exception as e:
        return f"예외: {str(e)}"

# 시스템 초기화 및 프로젝트 목록 확인
print("🏗️ 건축 도면 RAG 질의 시스템 초기화")
print("=" * 80)
print("📋 사용 가능한 프로젝트 목록:")
print(get_available_projects())

📋 사용 가능한 프로젝트 목록:
📋 .env 기반 설정 로드됨 - LLM: openai, 임베딩: openai
📊 LangSmith 추적 기능 로드됨
🔧 Multi-LLM Wrapper 로드됨
사용 가능한 프로젝트 컬렉션:
  - busanjangan_projectinfo: 61개 문서
  - busanjangan_projectdrawing: 4개 문서

📋 .env 기반 설정 로드됨 - LLM: openai, 임베딩: openai
📊 LangSmith 추적 기능 로드됨
🔧 Multi-LLM Wrapper 로드됨
사용 가능한 프로젝트 컬렉션:
  - busanjangan_projectinfo: 61개 문서
  - busanjangan_projectdrawing: 4개 문서



In [ ]:
# CLI 도구와 노트북 기능 통합

import re
from PIL import Image
import matplotlib.pyplot as plt

def enhanced_rag_query_with_images(query: str, project: str = None, n_results: int = 3):
    """
    CLI 도구로 RAG 질의를 수행하고, 관련 이미지를 노트북에서 표시합니다.
    
    Args:
        query (str): 검색할 질의 내용
        project (str): 특정 프로젝트명
        n_results (int): 반환할 검색 결과 수
    
    Returns:
        tuple: (CLI 출력 결과, 관련 이미지 경로 목록)
    """
    # CLI 도구로 RAG 질의 실행
    cli_result = run_rag_query(query, project=project, n_results=n_results)
    
    # CLI 결과에서 파일명 추출
    file_pattern = r'파일명:\s*([^\n]+)'
    file_matches = re.findall(file_pattern, cli_result)
    
    # 관련 이미지 파일 찾기
    related_images = []
    img_folder = "/Users/joguk/VLM/uploads/부산장안_프로젝트도면/"
    
    for file_match in file_matches:
        file_name = file_match.strip()
        # PDF 파일명에서 이미지 파일명 생성
        if file_name.endswith('.pdf'):
            image_name = file_name.replace('.pdf', '.png')
            image_path = os.path.join(img_folder, image_name)
            if os.path.exists(image_path):
                related_images.append(image_path)
    
    # 결과 출력
    print("🔍 CLI RAG 질의 결과:")
    print("=" * 80)
    print(cli_result)
    
    # 관련 이미지 표시
    if related_images:
        print(f"\n🖼️  관련 이미지 {len(related_images)}개 발견:")
        for i, img_path in enumerate(related_images[:3], 1):  # 최대 3개까지 표시
            print(f"  {i}. {os.path.basename(img_path)}")
            
            try:
                # 이미지 로드 및 표시
                img = Image.open(img_path)
                plt.figure(figsize=(12, 8))
                plt.imshow(img)
                plt.title(f"관련 도면 {i}: {os.path.basename(img_path)}", fontsize=14)
                plt.axis('off')
                plt.tight_layout()
                plt.show()
                
            except Exception as e:
                print(f"    ❌ 이미지 로드 실패: {e}")
    else:
        print("\n📝 관련 이미지를 찾을 수 없습니다.")
    
    return cli_result, related_images

def parse_cli_result_metadata(cli_output: str):
    """
    CLI 출력 결과에서 메타데이터를 파싱합니다.
    
    Args:
        cli_output (str): CLI 도구의 출력 결과
        
    Returns:
        list: 파싱된 검색 결과 메타데이터 목록
    """
    results = []
    
    # 결과 패턴 정의
    patterns = {
        'id': r'ID:\s*([^\n]+)',
        'similarity': r'유사도 \(거리\):\s*([^\n]+)',
        'project': r'프로젝트:\s*([^\n]+)', 
        'file_name': r'파일명:\s*([^\n]+)',
        'page': r'페이지:\s*([^\n]+)',
        'drawing_number': r'도면번호:\s*([^\n]+)',
        'drawing_title': r'도면제목:\s*([^\n]+)',
        'drawing_type': r'도면유형:\s*([^\n]+)'
    }
    
    # 결과별로 분할 (결과 N: 패턴으로)
    result_sections = re.split(r'\n결과 \d+:', cli_output)
    
    for section in result_sections[1:]:  # 첫 번째는 헤더이므로 제외
        result_data = {}
        
        for key, pattern in patterns.items():
            match = re.search(pattern, section)
            if match:
                result_data[key] = match.group(1).strip()
            else:
                result_data[key] = 'N/A'
        
        if result_data.get('id') != 'N/A':
            results.append(result_data)
    
    return results

# 테스트용 통합 질의 실행
test_query = "지하주차장 기초 구조평면도"
print(f"🔍 통합 RAG 질의 테스트: '{test_query}'")
print("=" * 80)

# 실행 예시 (주석 해제하여 사용)
# cli_output, images = enhanced_rag_query_with_images(test_query, project="부산장안_프로젝트도면")
# metadata = parse_cli_result_metadata(cli_output)
# print(f"\n📊 파싱된 메타데이터: {len(metadata)}개 결과")

print("다음 셀에서 통합 질의를 실행하려면:")
print("cli_output, images = enhanced_rag_query_with_images(test_query, project='부산장안_프로젝트도면')")
print("를 실행하세요.")

# 🚀 실제 RAG 질의 데모 실행

print("🏗️ 건축 도면 RAG 질의 데모")
print("=" * 80)

# 데모 1: 특정 프로젝트에서 이미지 포함 질의
demo_question = "지하주차장 기초 구조평면도에 대해 자세히 설명해주세요"
print(f"📋 데모 질문: {demo_question}")
print("🗂️  대상 프로젝트: 부산장안_프로젝트도면")
print("🖼️  이미지 표시: 활성화")
print("\n" + "=" * 80)

# 실제 질의 실행 (원하는 경우 주석 해제)
# simple_ask(demo_question, project="부산장안_프로젝트도면", show_images=True, n_results=2)

print("위 코드의 주석을 해제하여 데모를 실행하세요!")
print("\n💡 다른 질문 예시:")
print("- simple_ask('구조평면도의 종류를 알려주세요')")
print("- simple_ask('지하2층 주차장 구조는 어떻게 되나요?', project='부산장안_프로젝트도면')")
print("- simple_ask('프로젝트 전체 개요', show_images=False, n_results=5)")

🔍 통합 RAG 질의 테스트: '지하주차장 기초 구조평면도'
다음 셀에서 통합 질의를 실행하려면:
cli_output, images = enhanced_rag_query_with_images(test_query, project='부산장안_프로젝트도면')
를 실행하세요.


In [ ]:
# 🎯 올인원 RAG 질의 시스템 
# 사용자 친화적인 단일 인터페이스

import time
import re
import os
from PIL import Image
import matplotlib.pyplot as plt

def run_rag_query(query: str, project: str = None, n_results: int = 3):
    # 이 함수는 RAG 질의를 실행하고 결과를 반환합니다.
    # 실제 구현은 생략되었으며, 사용자는 이 부분에 자신의 로직을 추가해야 합니다.
    return "질의 결과 (텍스트 형식)"

def enhanced_rag_query_with_images(query: str, project: str = None, n_results: int = 3):
    """
    CLI 도구로 RAG 질의를 수행하고, 관련 이미지를 노트북에서 표시합니다.
    
    Args:
        query (str): 검색할 질의 내용
        project (str): 특정 프로젝트명
        n_results (int): 반환할 검색 결과 수
    
    Returns:
        tuple: (CLI 출력 결과, 관련 이미지 경로 목록)
    """
    # CLI 도구로 RAG 질의 실행
    cli_result = run_rag_query(query, project=project, n_results=n_results)
    
    # CLI 결과에서 파일명 추출
    file_pattern = r'파일명:\s*([^\n]+)'
    file_matches = re.findall(file_pattern, cli_result)
    
    # 관련 이미지 파일 찾기
    related_images = []
    img_folder = "/Users/joguk/VLM/uploads/부산장안_프로젝트도면/"
    
    for file_match in file_matches:
        file_name = file_match.strip()
        # PDF 파일명에서 이미지 파일명 생성
        if file_name.endswith('.pdf'):
            image_name = file_name.replace('.pdf', '.png')
            image_path = os.path.join(img_folder, image_name)
            if os.path.exists(image_path):
                related_images.append(image_path)
    
    # 결과 출력
    print("🔍 CLI RAG 질의 결과:")
    print("=" * 80)
    print(cli_result)
    
    # 관련 이미지 표시
    if related_images:
        print(f"\n🖼️  관련 이미지 {len(related_images)}개 발견:")
        for i, img_path in enumerate(related_images[:3], 1):  # 최대 3개까지 표시
            print(f"  {i}. {os.path.basename(img_path)}")
            
            try:
                # 이미지 로드 및 표시
                img = Image.open(img_path)
                plt.figure(figsize=(12, 8))
                plt.imshow(img)
                plt.title(f"관련 도면 {i}: {os.path.basename(img_path)}", fontsize=14)
                plt.axis('off')
                plt.tight_layout()
                plt.show()
                
            except Exception as e:
                print(f"    ❌ 이미지 로드 실패: {e}")
    else:
        print("\n📝 관련 이미지를 찾을 수 없습니다.")
    
    return cli_result, related_images

def simple_ask(question: str, project: str = None, show_images: bool = True, n_results: int = 3):
    """
    간단한 질문-답변 인터페이스
    
    Args:
        question (str): 질문
        project (str): 프로젝트명 (None이면 모든 프로젝트)
        show_images (bool): 관련 이미지 표시 여부
        n_results (int): 검색 결과 수
    """
    print(f"💬 질문: {question}")
    print(f"🗂️  대상: {project if project else '모든 프로젝트'}")
    print("=" * 80)
    
    start_time = time.time()
    
    if show_images:
        # 이미지 포함 통합 질의
        cli_output, images = enhanced_rag_query_with_images(question, project=project, n_results=n_results)
    else:
        # 텍스트만 질의
        cli_output = run_rag_query(question, project=project, n_results=n_results)
        print(cli_output)
    
    execution_time = time.time() - start_time
    print(f"\n⏱️  처리 시간: {execution_time:.2f}초")

# 📋 사용 예시들
print("🎯 올인원 RAG 질의 시스템 준비 완료!")
print("\n📋 사용 예시:")

examples = [
    {
        "title": "기본 질의 (모든 프로젝트)",
        "code": "simple_ask('구조평면도에 대해 알려주세요')"
    },
    {
        "title": "특정 프로젝트 질의 (이미지 포함)",
        "code": "simple_ask('지하주차장 배치는 어떻게 되나요?', project='부산장안_프로젝트도면')"
    },
    {
        "title": "텍스트만 질의 (이미지 제외)",
        "code": "simple_ask('화장실 배치도', project='부산장안_프로젝트도면', show_images=False)"
    },
    {
        "title": "많은 결과 요청",
        "code": "simple_ask('전체 도면 정보', n_results=10)"
    }
]

for i, example in enumerate(examples, 1):
    print(f"\n{i}. {example['title']}:")
    print(f"   {example['code']}")

print(f"\n🚀 바로 실행해보세요!")
print("=" * 80)

# 즉시 실행 가능한 테스트 (원하는 것을 주석 해제)
# simple_ask("지하주차장 기초 구조평면도에 대해 알려주세요", project="부산장안_프로젝트도면")
# simple_ask("구조평면도")
# simple_ask("화장실 배치", show_images=False)

🎯 올인원 RAG 질의 시스템 준비 완료!

📋 사용 예시:

1. 기본 질의 (모든 프로젝트):
   simple_ask('구조평면도에 대해 알려주세요')

2. 특정 프로젝트 질의 (이미지 포함):
   simple_ask('지하주차장 배치는 어떻게 되나요?', project='부산장안_프로젝트도면')

3. 텍스트만 질의 (이미지 제외):
   simple_ask('화장실 배치도', project='부산장안_프로젝트도면', show_images=False)

4. 많은 결과 요청:
   simple_ask('전체 도면 정보', n_results=10)

🚀 바로 실행해보세요!
